In [1]:
groups = {'G' :0 , 'A': 0, 'V': 0, 'L':0, 'I':0, 
         'S': 1, 'C':1, 'T':1, 'M':1,
         'P': 2,
         'F' : 3, 'W':3, 'Y':3,
         'H' :4 , 'K':4, 'R':4,
         'D':5, 'E' :5, 'N':5, 'Q' :5}

In [63]:
def sub_cost(aa1, aa2):
    if (aa1==aa2): return 0
    if (aa1 == 'X' or aa2=='X'): return 1
    if (groups[aa1] == groups[aa2]): return 1        
    return 2

In [82]:
def levenshtein(s1, s2):
    if len(s1) < len(s2):
        return levenshtein(s2, s1)

    # len(s1) >= len(s2)
    if len(s2) == 0:
        return len(s1)

    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1 # j+1 instead of j since previous_row and current_row are one character longer
            deletions = current_row[j] + 1       # than s2
            substitutions = previous_row[j] + sub_cost(c1, c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
    
    return previous_row[-1]

# def list_to_string(l):
#     s = ""
#     for c in l:
#         s+= c
#     return s

In [14]:
prolactin = ['M','N','I','K','G','S','P','W',
             'K','G','S','L','L','L','L','L',
             'V','S','N','L','L','L','C','Q',
             'S','V','A','P','L','P','I','C',
             'P','G','G','A','A','R','C','Q',
             'V','T','L','R','D','L','F','D',
             'R','A','V','V','L','S','H','Y',
             'I','H','N','L','S','S','E','M',
             'F','S','E','F','D','K','R','Y',
             'T','H','G','R','G','F','I','T',
             'K','A','I','N','S','C','H','T',
             'S','S','L','A','T','P','E','D',
             'K','E','Q','A','Q','Q','M','N',
             'Q','K','D','F','L','S','L','I',
             'V','S','I','L','R','S','W','N',
             'E','P','L','Y','H','L','V','T',
             'E','V','R','G','M','Q','E','A',
             'P','E','A','I','L','S','K','A',
             'V','E','I','E','E','Q','T','K',
             'R','L','L','E','G','M','E','L',
             'I','V','S','Q','V','H','P','E',
             'T','K','E','N','E','I','Y','P',
             'V','W','S','G','L','P','S','L',
             'Q','M','A','D','E','E','S','R',
             'L','S','A','Y','Y','N','L','L',
             'H','C','L','R','R','D','S','H',
             'K','I','D','N','Y','L','K','L',
             'L','K','C','R','I','I','H','N',
             'N','N','C',]

In [77]:
name_to_seq = {}
with open('proteins.fasta') as file:
    protopeptide = file.read()
    
    
    for protein in protopeptide.split(">"):
#         print(protein.replace('\n',''))
        splitted = protein.replace('\n','').split("]")
        if (splitted[0]):
            if (len(splitted) > 2):
                name = splitted[0] + "]" + splitted[1] + "]"
                seq = splitted[2]
            else:
                name = splitted[0]+"]"
                seq = splitted[1]
            
            name_to_seq[name] = list(seq.replace('\n','').replace(" ", ""))


In [84]:
def k_homologs(reference_sequence, k, sequence_dict):
    leven_dict = {}
    for name in sequence_dict:
        dist = levenshtein(reference_sequence, sequence_dict[name])
        leven_dict[name] = dist
    return leven_dict
    

In [86]:
leven_dict = k_homologs(prolactin, 10, name_to_seq)

In [97]:
import pandas as pd
homolog_df = pd.DataFrame.from_dict(leven_dict, 'index', ).sort_values(by=0)[:10].reset_index()
homolog_df.columns = ['proteinName', 'levenshteinDistance']

## Step 2.1

In [96]:
levenshtein(prolactin, ['P','I','D','N',
                        'Y','L','K','L','L',
                        'K','C','R','I','I',
                        'H','N','N','N','C'])

208

In [98]:
homolog_df

,proteinName,levenshteinDistance
0,NP_001157030.1 prolactin precursor [Homo sapiens],0
1,XP_011513056.1 PREDICTED: prolactin isoform X1...,1
2,XP_011513055.1 PREDICTED: prolactin isoform X1...,1
3,XP_002816520.2 PREDICTED: prolactin [Pongo abe...,4
4,XP_012364516.1 PREDICTED: prolactin [Nomascus ...,6
5,XP_016810474.1 PREDICTED: prolactin isoform X2...,6
6,XP_009448859.1 PREDICTED: prolactin isoform X2...,6
7,XP_008975038.1 PREDICTED: prolactin [Pan panis...,6
8,XP_003263607.1 PREDICTED: prolactin [Nomascus ...,6
9,XP_009448858.1 PREDICTED: prolactin isoform X1...,7
